In [1]:
from games_setup import *
from SBMLLint.common import constants as cn
from SBMLLint.common.molecule import Molecule, MoleculeStoichiometry
from SBMLLint.common.reaction import Reaction
from SBMLLint.games.som import SOM
from SBMLLint.common.simple_sbml import SimpleSBML

import collections
import itertools
import networkx as nx
import numpy as np
import pandas as pd
#
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
from SBMLLint.games.mesgraph import MESGraph
from SBMLLint.games.games_pp import GAMES_PP, SOMStoichiometry, SOMReaction, TOLERANCE
from SBMLLint.games.games_report import GAMESReport, SimplifiedReaction
from SBMLLint.games.games_report import NULL_STR, NUM_STAR, PARAGRAPH_DIVIDER, REPORT_DIVIDER

ReactionOperation = collections.namedtuple("ReactionOperation", 
    "reaction operation")

Current Directory: /Users/woosubshin/Desktop/ModelEngineering/SBMLLint/notebook


In [9]:
# Model 248 is for canceling error
# Model 14/167 for a type three error / echelon
# Model 7 for type 1 error
# Model 18 for type 2 error

######## examine model 628 for large echelon error ###########
simple = load_file_from_curated_data(7)
# for r in simple.reactions:
#   if r.category != cn.REACTION_BOUNDARY and r.category != cn.REACTION_1_1:
#     print(r.makeIdentifier(is_include_kinetics=False))
m = GAMES_PP(simple)
m.analyze()
gr = GAMESReport(m)

We just analyzed the data...
Type I error:  [PathComponents(node1='G2K', node2='G2R', reactions=['G2R_Creation']), PathComponents(node1='R', node2='G2R', reactions=['G2R_Creation']), PathComponents(node1='PG2', node2='PG2R', reactions=['PG2R_Creation']), PathComponents(node1='R', node2='PG2R', reactions=['PG2R_Creation']), PathComponents(node1='G1K', node2='G1R', reactions=['G1R_Binding']), PathComponents(node1='R', node2='G1R', reactions=['G1R_Binding'])]
Type II error:  []
Canceling error:  []
Echelon error:  []
Type III error:  []
Type I-SOM error:  set()


In [15]:
G2K = "G2K"
PG2R = "PG2R"
som = m.getNode(simple.getMolecule("G2K"))
gr.getMoleculeEqualityPath(som, "G2K", "PG2R")

[PathComponents(node1='G2K', node2='PG2', reactions=['Cdc2Phos']),
 PathComponents(node1='PG2', node2='PG2R', reactions=['Rum1DegInPG2R'])]

In [19]:
count, report = gr.getMoleculeEqualityPathReport(G2K, PG2R, 0, explain_details=False)
pri
report

'1. Cdc2Phos: G2K -> PG2\n2. Rum1DegInPG2R: PG2R -> PG2\n'

In [5]:
# From model 248
canceling_report, error_num = gr.reportCancelingError(m.canceling_errors, explain_details=True)
canceling_report

'We detected a mass imbalance from the following reactions:\n\n1. OxidativePhosphorylation: 6.00 ADP + CTtis -> 6.00 ATP\n\n2. ATPase: ATP -> ADP\n\n*ATP and ADP have the same mass according to the above reaction\n\n\nTherefore, they will result in empty product with zero mass:\n\nOxidativePhosphorylation: CTtis -> \n\nThis indicates a mass conflict between reactions.\n\n----------------------------------------------------------------------\n\n----------------------------------------------------------------------\n\n\n\n**********************************************************************\n\n'

In [6]:
correct_report = NULL_STR
correct_report = correct_report + "We detected a mass imbalance from the following reactions:\n\n"
correct_report = correct_report + "1. OxidativePhosphorylation: 6.00 ADP + CTtis -> 6.00 ATP\n\n"
correct_report = correct_report + "2. ATPase: ATP -> ADP\n\n"
correct_report = correct_report + "*ATP and ADP have the same mass according to the above reaction\n\n\n"
correct_report = correct_report + "Therefore, they will result in empty product with zero mass:\n\n"
correct_report = correct_report + "OxidativePhosphorylation: CTtis -> \n\n"
correct_report = correct_report + "This indicates a mass conflict between reactions."
correct_report = correct_report + "\n%s%s\n" % (PARAGRAPH_DIVIDER, PARAGRAPH_DIVIDER)
correct_report + correct_report + "\n%s\n" % REPORT_DIVIDER

correct_report

'We detected a mass imbalance from the following reactions:\n\n1. OxidativePhosphorylation: 6.00 ADP + CTtis -> 6.00 ATP\n\n2. ATPase: ATP -> ADP\n\n*ATP and ADP have the same mass according to the above reaction\n\n\nTherefore, they will result in empty product with zero mass:\n\nOxidativePhosphorylation: CTtis -> \n\nThis indicates a mass conflict between reactions.\n\n----------------------------------------------------------------------\n\n----------------------------------------------------------------------\n\n'

In [7]:
simple.reactions

[ATPase: ATP -> ADP; Tissue * Katpase * ATP,
 OxidativePhosphorylation: 6.00 ADP + CTtis -> 6.00 ATP; Tissue * Vmax * (CFtis / (Km + CFtis)) * ADP / (Kadp + ADP),
 ConvectionTransport:  -> CTcap; Qm * (CTart - CTcap),
 DiffusionTransport: CTcap -> CTtis; PSm * (CFcap - CFtis),
 CreatineKinase: ADP + PCr -> ATP + Cr; Tissue * (VfCK * ADP * PCr / (Kb * Kia) - VrCK * Cr * ATP / (Kiq * Kp)) / ((Kia + ADP) / Kia + ATP / Kiq + PCr / Kib + ADP * PCr / (Kb * Kia) + Cr * ATP / (Kiq * Kp))]

In [8]:
m.nodes

NodeView(({PCr}, {Cr}, {ADP=ATP}, {CTcap=CTtis}))